In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###We're using the audio and text files generated by the Whisper Library

In [ ]:
audio_path='/content/drive/MyDrive/Techolution Final/one.wav'
text_path='/content/drive/MyDrive/Techolution Final/one.txt'

#Silence time Analysis from audio

Librosa is an audio-processing library. We begin by loading the audio data and extracting the sample rate for further processing. A critical component of the code is setting a threshold value, denoted as 'threshold,' which acts as the basis for identifying silence in the audio. The threshold can be adjusted to suit the specific characteristics of the audio file. The code then iterates through the audio data, segmenting it into periods of silence and non-silence based on the amplitude's comparison to the threshold. It effectively detects transitions from non-silence to silence and vice versa.

As the code progresses, it accumulates these silence periods within the 'silence_periods' list. Importantly, it calculates the duration of each detected silence period and prints it out if it exceeds a threshold of 3 seconds. Moreover, the code keeps a running total of the duration of prolonged silence periods in the 'count' variable. This information can be valuable for tasks such as audio transcription or speech analysis. Lastly, the code wraps up by determining the total length of the audio file in seconds, making use of Librosa's 'get_duration' function.

In [ ]:
import librosa
import numpy as np

count = 0
# Load the audio data and sample rate
audio_path = '/content/drive/MyDrive/Techolution Final/one.wav'
audio, sample_rate = librosa.load(audio_path)

# Set a threshold for silence detection (adjust this value as needed)
threshold = 0.4  # You may need to experiment with different values

# Find silence periods
silence_periods = []
is_silent = True
current_period = []
for amplitude in audio:
    if abs(amplitude) < threshold:
        if not is_silent:
            # Transition to silence
            silence_periods.append(current_period)
            current_period = []
            is_silent = True
        current_period.append(amplitude)
    else:
        if is_silent:
            # Transition from silence
            current_period.append(amplitude)
            is_silent = False
        else:
            current_period.append(amplitude)

# Append any remaining silent period
if is_silent:
    silence_periods.append(current_period)

# Analyze or process the detected silence periods as needed
for i, period in enumerate(silence_periods):
    duration = len(period) / float(sample_rate)
    if(duration > 3.00):
      count+=duration
      print(f"Silence Period {i + 1}: Duration {duration:.2f} seconds")
print(count)
audio_length_seconds = librosa.get_duration(y=audio, sr=sample_rate)
audio_length_seconds

Silence Period 21: Duration 4.23 seconds
Silence Period 32: Duration 3.06 seconds
Silence Period 360: Duration 3.54 seconds
Silence Period 398: Duration 3.99 seconds
Silence Period 823: Duration 3.03 seconds
Silence Period 848: Duration 3.97 seconds
Silence Period 853: Duration 7.08 seconds
Silence Period 870: Duration 4.42 seconds
Silence Period 1415: Duration 6.62 seconds
Silence Period 1658: Duration 3.38 seconds
43.319319727891155


155.54

###The below metric is used to quantify how confident an interviewee is, based on the silence that was observed during their interview. It's a simple ratio that uses the silence time divided by the total time.


In [ ]:
underconfidence_metric = count/audio_length_seconds
underconfidence_metric

confidence_metric = 1-underconfidence_metric
confidence_metric

0.7214908079729256

#Emotion Analysis from Transcript
We read the text from the .txt file

In [ ]:
# Specify the path to the text file you want to read
file_path = "/content/drive/MyDrive/Techolution Final/one.txt"  # Replace with your file path

# Open the file in read mode (default)
with open(file_path, "r") as file:
    transcript = file.read()

# Now, the file contents are stored in the 'file_contents' variable as a string

# Print or use the string as needed
print(transcript)

So, my name is Dipanshu and I belong to ISA, which is in Haryana state and I've done my
bachelor's from Vyamsi University of Science and Technology, which is in Paridabad.
And so for the last 1.5 years and being the part of the ISQ organization as a Python developer.
So during my tenure, I've been into many projects.
So the main project I'm handling is the analytics.
So in this project, I used to develop APIs for the admin users so that they can see
the business insights of their business, business insights.
So these API, these are the rest APIs, which are developed in the Django framework.
And these API include the total sales, total orders.
So how much the card payment, how much the online payment, so how much the cash payment.
So these include all the stats for the business insights.
And apart from that, I'm also handling one other clone, which is a fancy only fence clone.
So in that project, I'm handling the whole admin panel and the posting service.
So apart from that, I'm also ha

###We perform Sentiment analysis using the VADER (Valence Aware Dictionary and sentiment Reasoner) sentiment analysis tool from the NLTK (Natural Language Toolkit) library.

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

#VADER is a lexicon and rule-based sentiment analysis tool that assigns sentiment scores to text, helping determine whether the text expresses a positive, negative, or neutral sentiment.

Following the initialization, the code analyzes the sentiment of a text or transcript, which should be stored in the 'transcript' variable. The sentiment analysis process computes various sentiment scores, including a 'compound' score, which represents an overall sentiment score for the text.

The code then interprets the sentiment scores to assign a sentiment label. If the 'compound' score is greater than or equal to 0.05, it categorizes the sentiment as "Positive." Conversely, if the 'compound' score is less than or equal to -0.05, it categorizes the sentiment as "Negative." If the 'compound' score falls between these thresholds, the code labels the sentiment as "Neutral."

Lastly, the code prints out the sentiment analysis results, displaying both the sentiment scores, which provide a more granular view of positive, negative, and neutral sentiment, and the sentiment label, which simplifies the overall sentiment classification.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer


# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Analyze sentiment
sentiment_scores = analyzer.polarity_scores(transcript)

# Interpret the sentiment scores
sentiment_label = ""
if sentiment_scores["compound"] >= 0.05:
    sentiment_label = "Positive"
elif sentiment_scores["compound"] <= -0.05:
    sentiment_label = "Negative"
else:
    sentiment_label = "Neutral"

# Print the sentiment analysis results
print("Sentiment Scores:", sentiment_scores)
print("Sentiment Label:", sentiment_label)

Sentiment Scores: {'neg': 0.0, 'neu': 0.922, 'pos': 0.078, 'compound': 0.9742}
Sentiment Label: Positive


#Number of Filler words
The code takes the input text and converts it to lowercase to ensure case-insensitive matching. It then defines a list of common filler words such as ***"the," "and," "in,"*** and others. These filler words are often used in speech and writing but may not carry substantial meaning by themselves.

Next, the code creates a regular expression pattern, 'filler_pattern,' that matches any of the defined filler words when they appear as whole words within the text. The 're.findall' function is used to find all instances of these filler words within the input text, taking into account case-insensitivity.

The code counts the number of filler word occurrences and stores this count in the 'num_fillers' variable. Additionally, it retains the original text in the 'text' variable.

After counting the filler words, the code proceeds to split the 'text' into individual words, using whitespace as a delimiter. This allows it to calculate the total word count, which it stores in the 'word_count' variable.

Finally, the code prints out the number of filler words found in the text and the total word count.

In [ ]:
import re

def count_fillers(input_text):
    # Convert all characters to lowercase
    input_text = input_text.lower()

    # Define a list of common filler words
    filler_words = ["the", "and", "in", "to", "of", "with", "on", "for", "a", "an", "so"]

    # Create a regular expression pattern to match filler words (case-insensitive)
    filler_pattern = r'\b(?:' + '|'.join(filler_words) + r')\b'

    # Use re.findall to find all occurrences of filler words
    filler_matches = re.findall(filler_pattern, input_text, flags=re.IGNORECASE)

    # Count the number of filler words
    num_fillers = len(filler_matches)

    return num_fillers,input_text


# Count the number of filler words in the text
num_fillers,text = count_fillers(transcript)

# Print the count
print("Number of filler words:", num_fillers)
print(text)


words = text.split()

# Count the number of words
word_count = len(words)

# Print the word count
print("Number of words:", word_count)

Number of filler words: 92
so, my name is dipanshu and i belong to isa, which is in haryana state and i've done my
bachelor's from vyamsi university of science and technology, which is in paridabad.
and so for the last 1.5 years and being the part of the isq organization as a python developer.
so during my tenure, i've been into many projects.
so the main project i'm handling is the analytics.
so in this project, i used to develop apis for the admin users so that they can see
the business insights of their business, business insights.
so these api, these are the rest apis, which are developed in the django framework.
and these api include the total sales, total orders.
so how much the card payment, how much the online payment, so how much the cash payment.
so these include all the stats for the business insights.
and apart from that, i'm also handling one other clone, which is a fancy only fence clone.
so in that project, i'm handling the whole admin panel and the posting service.
so a

###The below code hels us find the percentage of words in the interview that were filler, thereby telling us how much useful the interview was in conveying the candidate's skillsets.

In [ ]:
metric_filler_percentage=num_fillers/word_count
metric_filler_percentage

metric_canon_percentage = 1 - metric_filler_percentage
metric_canon_percentage

0.7430167597765363

In [ ]:
Metric_Communication = metric_canon_percentage*0.15
Metric_Communication

0.11145251396648044